# Thesis Experiment Notebook (Part A + Part B MVP)

This notebook runs a minimal end-to-end experiment for finite-sample Sharpe inference under dependence/heavy tails.

Scope:
- Part A: Monte Carlo calibration on 3 DGPs (`iid_normal`, standardized `iid_t_nu`, `garch11_t`) for `n in {120, 240, 1200}` and `S_true in {0.0, 0.5}`.
- Part B (MVP): empirical PIT adequacy on holdout disjoint windows at `n=240` for 3 fitted models.

Produced files (saved under `outputs/thesis_mvp/`):
- `results_partA.parquet` (or CSV fallback)
- `results_partB_pit.parquet` (or CSV fallback)
- `model_fit_summary.json`
- `environment_versions.json` and `.txt`
- Figures (`.png` + `.pdf`) in `outputs/thesis_mvp/figures/`

In [ ]:
import json
import hashlib
import platform
import random
import sys
from pathlib import Path
import importlib.metadata as ilmd

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(iterable=None, *args, **kwargs):
        return iterable if iterable is not None else []
    print("tqdm not installed; using no-op progress wrapper.")
from scipy import stats

plt.style.use("ggplot")
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

RUN_ROOT = Path("outputs/thesis_mvp").resolve()
RUN_ROOT.mkdir(parents=True, exist_ok=True)

SAVED_ARTIFACTS = []


def _pkg_version(name: str) -> str:
    try:
        return ilmd.version(name)
    except Exception:
        return "not-installed"


version_payload = {
    "python": sys.version,
    "platform": platform.platform(),
    "packages": {
        "numpy": _pkg_version("numpy"),
        "pandas": _pkg_version("pandas"),
        "scipy": _pkg_version("scipy"),
        "matplotlib": _pkg_version("matplotlib"),
        "tqdm": _pkg_version("tqdm"),
        "arch": _pkg_version("arch"),
    },
}

print("Environment versions:")
print(json.dumps(version_payload, indent=2))

versions_json_path = RUN_ROOT / "environment_versions.json"
versions_txt_path = RUN_ROOT / "environment_versions.txt"
versions_json_path.write_text(json.dumps(version_payload, indent=2), encoding="utf-8")
versions_txt_path.write_text(json.dumps(version_payload, indent=2), encoding="utf-8")

SAVED_ARTIFACTS.extend([str(versions_json_path), str(versions_txt_path)])


In [ ]:
CONFIG = {
    "R": 5000,
    "seed": 12345,
    "n_list": [120, 240, 1200],
    "S_true_list": [0.0, 0.5],
    "alpha": 0.05,
    "burn_B": 500,
    "t_df": 5.0,
    "garch_alpha": 0.05,
    "garch_beta": 0.90,
    "DSR_M": None,
    "cache_dir": str((RUN_ROOT / "cache").resolve()),
    "figures_dir": str((RUN_ROOT / "figures").resolve()),
    "data_path": "data/monthly_excess_returns.csv",
}

if CONFIG["t_df"] <= 4:
    raise ValueError("CONFIG['t_df'] must be > 4 for finite higher moments used by inference methods.")
if CONFIG["garch_alpha"] + CONFIG["garch_beta"] >= 1:
    raise ValueError("Need garch_alpha + garch_beta < 1 for unconditional variance stationarity.")

Path(CONFIG["cache_dir"]).mkdir(parents=True, exist_ok=True)
Path(CONFIG["figures_dir"]).mkdir(parents=True, exist_ok=True)

print("Config:")
print(json.dumps(CONFIG, indent=2))

In [ ]:
# Add project root to sys.path and import existing project modules.
cwd = Path.cwd().resolve()
PROJECT_ROOT = None
for p in [cwd, *cwd.parents]:
    if (p / "src" / "sharpe_mc.py").exists():
        PROJECT_ROOT = p
        if str(p) not in sys.path:
            sys.path.insert(0, str(p))
        break

if PROJECT_ROOT is None:
    raise RuntimeError("Could not locate project root containing src/sharpe_mc.py")

from src import sharpe_mc

PROJECT_NOTES = []

required_symbols = [
    "simulate_iid_normal",
    "simulate_iid_t5",
    "simulate_garch11_t5",
    "sharpe_ratio",
    "se_naive",
    "se_hac",
    "psr_probability",
]
missing = [name for name in required_symbols if not hasattr(sharpe_mc, name)]
if missing:
    raise RuntimeError(f"Missing required symbols in src/sharpe_mc.py: {missing}")

if not hasattr(sharpe_mc, "dsr_probability"):
    PROJECT_NOTES.append("DSR function missing in project; DSR will be skipped.")

print(f"Project root: {PROJECT_ROOT}")
if PROJECT_NOTES:
    print("Project notes:")
    for note in PROJECT_NOTES:
        print("-", note)

In [ ]:
CACHE_DIR = Path(CONFIG["cache_dir"])
FIGURES_DIR = Path(CONFIG["figures_dir"])
RESULTS_DIR = RUN_ROOT

OUTPUT_COLS = [
    "dgp",
    "n",
    "S_true",
    "method",
    "bias",
    "rmse",
    "coverage_95",
    "reject_rate_H0_S_le_0",
    "se_ratio",
    "psr_reject_rate",
    "dsr_reject_rate",
]


def _track_artifact(path: Path) -> Path:
    path = Path(path).resolve()
    SAVED_ARTIFACTS.append(str(path))
    return path


def set_global_seed(seed: int) -> None:
    random.seed(int(seed))
    np.random.seed(int(seed))


def _safe_tag(x: float) -> str:
    return str(x).replace("-", "m").replace(".", "p")


def series_checksum(arr: np.ndarray) -> str:
    a = np.asarray(arr, dtype=np.float64)
    return hashlib.sha256(a.tobytes()).hexdigest()[:12]


def cache_save_df(df: pd.DataFrame, stem: str) -> Path:
    stem = stem.strip()
    pq = CACHE_DIR / f"{stem}.parquet"
    csv = CACHE_DIR / f"{stem}.csv"
    try:
        df.to_parquet(pq, index=False)
        return pq
    except Exception:
        df.to_csv(csv, index=False)
        return csv


def cache_load_df(stem: str) -> pd.DataFrame | None:
    stem = stem.strip()
    pq = CACHE_DIR / f"{stem}.parquet"
    csv = CACHE_DIR / f"{stem}.csv"
    if pq.exists():
        try:
            return pd.read_parquet(pq)
        except Exception:
            pass
    if csv.exists():
        return pd.read_csv(csv)
    return None


def result_save_df(df: pd.DataFrame, base_name: str) -> list[Path]:
    out_paths: list[Path] = []
    pq = RESULTS_DIR / f"{base_name}.parquet"
    csv = RESULTS_DIR / f"{base_name}.csv"
    try:
        df.to_parquet(pq, index=False)
        out_paths.append(_track_artifact(pq))
    except Exception:
        pass
    df.to_csv(csv, index=False)
    out_paths.append(_track_artifact(csv))
    return out_paths


def cache_save_array(arr: np.ndarray, stem: str) -> Path:
    path = CACHE_DIR / f"{stem}.npy"
    np.save(path, np.asarray(arr, dtype=np.float64))
    return path


def cache_load_array(stem: str) -> np.ndarray | None:
    path = CACHE_DIR / f"{stem}.npy"
    if path.exists():
        return np.load(path)
    return None


def save_figure(fig: plt.Figure, stem: str) -> tuple[Path, Path]:
    png_path = FIGURES_DIR / f"{stem}.png"
    pdf_path = FIGURES_DIR / f"{stem}.pdf"
    fig.savefig(png_path, dpi=160, bbox_inches="tight")
    fig.savefig(pdf_path, bbox_inches="tight")
    _track_artifact(png_path)
    _track_artifact(pdf_path)
    return png_path, pdf_path


def run_methods_on_sample(sample: np.ndarray, S_true: float, n: int) -> dict:
    """Run all requested inference methods on one sample and return per-method scalars."""
    x = np.asarray(sample, dtype=np.float64)
    alpha = float(CONFIG["alpha"])
    z_two = float(stats.norm.ppf(1.0 - alpha / 2.0))
    z_one = float(stats.norm.ppf(1.0 - alpha))

    S_hat, _, _ = sharpe_mc.sharpe_ratio(x)

    se_iid = float(sharpe_mc.se_naive(S_hat, n))
    ci_iid_low = float(S_hat - z_two * se_iid)
    ci_iid_high = float(S_hat + z_two * se_iid)
    cover_iid = bool(ci_iid_low <= S_true <= ci_iid_high)
    reject_iid = bool((S_hat / se_iid) > z_one)

    se_hac = float(sharpe_mc.se_hac(x, S_hat))
    ci_hac_low = float(S_hat - z_two * se_hac)
    ci_hac_high = float(S_hat + z_two * se_hac)
    cover_hac = bool(ci_hac_low <= S_true <= ci_hac_high)
    reject_hac = bool((S_hat / se_hac) > z_one)

    skew_x = float(stats.skew(x, bias=False))
    kurt_x = float(stats.kurtosis(x, fisher=False, bias=False))

    psr_prob = float(sharpe_mc.psr_probability(S_hat, skew_x, kurt_x, n, S_ref=0.0))
    psr_reject = bool(psr_prob > (1.0 - alpha))

    dsr_prob = np.nan
    dsr_reject = np.nan
    if CONFIG["DSR_M"] is not None and hasattr(sharpe_mc, "dsr_probability"):
        dsr_prob = float(
            sharpe_mc.dsr_probability(
                S_hat,
                skew_x,
                kurt_x,
                n,
                int(CONFIG["DSR_M"]),
                S_ref=0.0,
            )
        )
        dsr_reject = bool(dsr_prob > (1.0 - alpha))

    return {
        "S_hat": float(S_hat),
        "iid_normal_analytic": {
            "ci_low": ci_iid_low,
            "ci_high": ci_iid_high,
            "coverage": cover_iid,
            "reject": reject_iid,
            "se": se_iid,
        },
        "hac_newey_west": {
            "ci_low": ci_hac_low,
            "ci_high": ci_hac_high,
            "coverage": cover_hac,
            "reject": reject_hac,
            "se": se_hac,
        },
        "psr": {
            "ci_low": np.nan,
            "ci_high": np.nan,
            "coverage": np.nan,
            "reject": psr_reject,
            "se": np.nan,
            "prob": psr_prob,
        },
        "dsr": {
            "ci_low": np.nan,
            "ci_high": np.nan,
            "coverage": np.nan,
            "reject": dsr_reject,
            "se": np.nan,
            "prob": dsr_prob,
        },
    }


def simulate_partA_cell(dgp: str, n: int, S_true: float, R: int, seed: int) -> pd.DataFrame:
    """Simulate one (dgp, n, S_true) cell and store per-replication scalars only."""
    set_global_seed(seed)
    rng = np.random.default_rng(seed + 100003)

    S_hat = np.empty(R, dtype=np.float64)

    se_iid = np.empty(R, dtype=np.float64)
    cover_iid = np.empty(R, dtype=bool)
    reject_iid = np.empty(R, dtype=bool)

    se_hac = np.empty(R, dtype=np.float64)
    cover_hac = np.empty(R, dtype=bool)
    reject_hac = np.empty(R, dtype=bool)

    psr_reject = np.empty(R, dtype=bool)
    dsr_reject = np.full(R, np.nan, dtype=np.float64)

    ci_iid_low = np.empty(R, dtype=np.float64)
    ci_iid_high = np.empty(R, dtype=np.float64)
    ci_hac_low = np.empty(R, dtype=np.float64)
    ci_hac_high = np.empty(R, dtype=np.float64)

    for i in tqdm(range(R), desc=f"{dgp} | n={n} | S={S_true}", leave=False):
        if dgp == "iid_normal":
            sample = sharpe_mc.simulate_iid_normal(n=n, mu=S_true, sigma=1.0, rng=rng)
        elif dgp == "iid_t":
            sample = sharpe_mc.simulate_iid_t5(
                n=n,
                mu=S_true,
                sigma=1.0,
                rng=rng,
                df=int(CONFIG["t_df"]),
            )
        elif dgp == "garch11_t":
            sample = sharpe_mc.simulate_garch11_t5(
                n=n,
                mu=S_true,
                sigma=1.0,
                alpha=float(CONFIG["garch_alpha"]),
                beta=float(CONFIG["garch_beta"]),
                df=int(CONFIG["t_df"]),
                burn=int(CONFIG["burn_B"]),
            )
        else:
            raise ValueError(f"Unknown dgp: {dgp}")

        out = run_methods_on_sample(sample=sample, S_true=S_true, n=n)
        S_hat[i] = out["S_hat"]

        iid = out["iid_normal_analytic"]
        se_iid[i] = iid["se"]
        cover_iid[i] = iid["coverage"]
        reject_iid[i] = iid["reject"]
        ci_iid_low[i] = iid["ci_low"]
        ci_iid_high[i] = iid["ci_high"]

        hac = out["hac_newey_west"]
        se_hac[i] = hac["se"]
        cover_hac[i] = hac["coverage"]
        reject_hac[i] = hac["reject"]
        ci_hac_low[i] = hac["ci_low"]
        ci_hac_high[i] = hac["ci_high"]

        psr_reject[i] = out["psr"]["reject"]

        if CONFIG["DSR_M"] is not None and hasattr(sharpe_mc, "dsr_probability"):
            dsr_reject[i] = float(out["dsr"]["reject"])

    return pd.DataFrame(
        {
            "S_hat": S_hat,
            "se_iid": se_iid,
            "se_hac": se_hac,
            "cover_iid": cover_iid,
            "cover_hac": cover_hac,
            "reject_iid": reject_iid,
            "reject_hac": reject_hac,
            "psr_reject": psr_reject,
            "dsr_reject": dsr_reject,
            "ci_iid_low": ci_iid_low,
            "ci_iid_high": ci_iid_high,
            "ci_hac_low": ci_hac_low,
            "ci_hac_high": ci_hac_high,
        }
    )


def summarize_results_partA(rep_df: pd.DataFrame, dgp: str, n: int, S_true: float) -> pd.DataFrame:
    """Aggregate one cell to required output schema.

    bias and rmse are computed once from S_hat and replicated across method rows.
    """
    s = rep_df["S_hat"].to_numpy(dtype=np.float64)
    bias = float(np.mean(s) - S_true)
    rmse = float(np.sqrt(np.mean((s - S_true) ** 2)))

    sd_emp = float(np.std(s, ddof=1))

    psr_rate = float(np.mean(rep_df["psr_reject"])) if len(rep_df) else np.nan
    dsr_available = bool(np.isfinite(rep_df["dsr_reject"]).any())
    dsr_rate = float(np.nanmean(rep_df["dsr_reject"])) if dsr_available else np.nan

    se_ratio_iid = float(np.mean(rep_df["se_iid"]) / sd_emp) if sd_emp > 0 else np.nan
    se_ratio_hac = float(np.mean(rep_df["se_hac"]) / sd_emp) if sd_emp > 0 else np.nan

    rows = [
        {
            "dgp": dgp,
            "n": int(n),
            "S_true": float(S_true),
            "method": "iid_normal_analytic",
            "bias": bias,
            "rmse": rmse,
            "coverage_95": float(np.mean(rep_df["cover_iid"])),
            "reject_rate_H0_S_le_0": float(np.mean(rep_df["reject_iid"])),
            "se_ratio": se_ratio_iid,
            "psr_reject_rate": psr_rate,
            "dsr_reject_rate": dsr_rate,
        },
        {
            "dgp": dgp,
            "n": int(n),
            "S_true": float(S_true),
            "method": "hac_newey_west",
            "bias": bias,
            "rmse": rmse,
            "coverage_95": float(np.mean(rep_df["cover_hac"])),
            "reject_rate_H0_S_le_0": float(np.mean(rep_df["reject_hac"])),
            "se_ratio": se_ratio_hac,
            "psr_reject_rate": psr_rate,
            "dsr_reject_rate": dsr_rate,
        },
        {
            "dgp": dgp,
            "n": int(n),
            "S_true": float(S_true),
            "method": "psr",
            "bias": bias,
            "rmse": rmse,
            "coverage_95": np.nan,
            "reject_rate_H0_S_le_0": psr_rate,
            "se_ratio": np.nan,
            "psr_reject_rate": psr_rate,
            "dsr_reject_rate": dsr_rate,
        },
        {
            "dgp": dgp,
            "n": int(n),
            "S_true": float(S_true),
            "method": "dsr",
            "bias": bias,
            "rmse": rmse,
            "coverage_95": np.nan,
            "reject_rate_H0_S_le_0": dsr_rate if dsr_available else np.nan,
            "se_ratio": np.nan,
            "psr_reject_rate": psr_rate,
            "dsr_reject_rate": dsr_rate,
        },
    ]

    return pd.DataFrame(rows, columns=OUTPUT_COLS)

In [ ]:
# Part A: run (with cache reuse if present)
PARTA_DGPS = ["iid_normal", "iid_t", "garch11_t"]
R = int(CONFIG["R"])

partA_grid = [
    (dgp, int(n), float(s_true))
    for dgp in PARTA_DGPS
    for n in CONFIG["n_list"]
    for s_true in CONFIG["S_true_list"]
]

summary_frames: list[pd.DataFrame] = []
partA_cell_cache = []

if CONFIG["DSR_M"] is None:
    print("DSR skipped by default (CONFIG['DSR_M']=None); DSR metrics are filled with NaN.")

for idx, (dgp, n, S_true) in enumerate(tqdm(partA_grid, desc="Part A cells")):
    cell_seed = int(CONFIG["seed"] + 1009 * (idx + 1))
    cache_stem = (
        f"partA_scalars_{dgp}_n{n}_S{_safe_tag(S_true)}_"
        f"R{R}_seed{cell_seed}_nu{_safe_tag(CONFIG['t_df'])}_"
        f"a{_safe_tag(CONFIG['garch_alpha'])}_b{_safe_tag(CONFIG['garch_beta'])}_"
        f"burn{int(CONFIG['burn_B'])}_alpha{_safe_tag(CONFIG['alpha'])}_"
        f"dsr{CONFIG['DSR_M']}"
    )

    rep_df = cache_load_df(cache_stem)
    if rep_df is None:
        rep_df = simulate_partA_cell(
            dgp=dgp,
            n=n,
            S_true=S_true,
            R=R,
            seed=cell_seed,
        )
        cache_path = cache_save_df(rep_df, cache_stem)
    else:
        cache_path = CACHE_DIR / (cache_stem + (".parquet" if (CACHE_DIR / f"{cache_stem}.parquet").exists() else ".csv"))

    partA_cell_cache.append(str(cache_path))
    summary_frames.append(summarize_results_partA(rep_df=rep_df, dgp=dgp, n=n, S_true=S_true))

results_partA = pd.concat(summary_frames, ignore_index=True)[OUTPUT_COLS]
results_partA


In [ ]:
# Part A: save result table artifacts
partA_paths = result_save_df(results_partA, "results_partA")
print("Saved Part A results:")
for p in partA_paths:
    print("-", p)

print("\nPart A head:")
print(results_partA.head(12))


In [ ]:
# Part A sanity checks
sanity_rows = []

# 1) iid normal coverage near 95% at n=1200, S_true=0
cov_row = results_partA[
    (results_partA["dgp"] == "iid_normal")
    & (results_partA["n"] == 1200)
    & (results_partA["S_true"] == 0.0)
    & (results_partA["method"] == "iid_normal_analytic")
]
if len(cov_row) == 1:
    cov_val = float(cov_row.iloc[0]["coverage_95"])
    cov_ok = abs(cov_val - 0.95) <= 0.02
    sanity_rows.append({"check": "iid normal coverage @ n=1200", "value": cov_val, "pass": cov_ok})
else:
    sanity_rows.append({"check": "iid normal coverage @ n=1200", "value": np.nan, "pass": False})

# 2) standardized t innovations variance close to 1
rng_t = np.random.default_rng(int(CONFIG["seed"]) + 555)
big_t = sharpe_mc.simulate_iid_t5(
    n=200_000,
    mu=0.0,
    sigma=1.0,
    rng=rng_t,
    df=int(CONFIG["t_df"]),
)
var_t = float(np.var(big_t, ddof=1))
var_ok = abs(var_t - 1.0) <= 0.08
sanity_rows.append({"check": "standardized t var ~= 1", "value": var_t, "pass": var_ok})

# 3) GARCH variance recursion positivity
set_global_seed(int(CONFIG["seed"]) + 777)
nu = float(CONFIG["t_df"])
alpha_g = float(CONFIG["garch_alpha"])
beta_g = float(CONFIG["garch_beta"])
omega_g = 1.0 - alpha_g - beta_g
total = 2000
z = np.random.standard_t(df=nu, size=total) / np.sqrt(nu / (nu - 2.0))
h = np.empty(total, dtype=np.float64)
eps_prev = 0.0
h_prev = 1.0
for t in range(total):
    h_t = omega_g + alpha_g * (eps_prev ** 2) + beta_g * h_prev
    h_t = max(h_t, 1e-14)
    eps_t = np.sqrt(h_t) * z[t]
    h[t] = h_t
    h_prev = h_t
    eps_prev = eps_t

min_h = float(np.min(h))
sanity_rows.append({"check": "garch sigma_t^2 > 0", "value": min_h, "pass": bool(min_h > 0.0)})

sanity_df = pd.DataFrame(sanity_rows)
print(sanity_df)

In [ ]:
# Part A plots: coverage vs n by method for each DGP
method_order = ["iid_normal_analytic", "hac_newey_west", "psr", "dsr"]

for dgp in PARTA_DGPS:
    s_values = list(CONFIG["S_true_list"])
    fig, axes = plt.subplots(1, len(s_values), figsize=(6 * len(s_values), 4), sharey=True)
    if len(s_values) == 1:
        axes = [axes]

    for ax, s_true in zip(axes, s_values):
        sub = results_partA[(results_partA["dgp"] == dgp) & (results_partA["S_true"] == float(s_true))]
        for method in method_order:
            s = sub[sub["method"] == method].sort_values("n")
            y = s["coverage_95"].to_numpy(dtype=float)
            if np.isfinite(y).any():
                ax.plot(s["n"], y, marker="o", label=method)

        ax.axhline(0.95, color="black", linestyle="--", linewidth=1.0, alpha=0.8)
        ax.set_title(f"{dgp} | S_true={s_true}")
        ax.set_xlabel("n")
        ax.set_ylabel("coverage_95")
        ax.set_xticks(CONFIG["n_list"])
        ax.set_ylim(0.0, 1.0)

    handles, labels = axes[0].get_legend_handles_labels()
    if handles:
        fig.legend(handles, labels, loc="upper center", ncol=min(4, len(labels)))
    fig.suptitle(f"Part A Coverage vs n ({dgp})", y=1.05)
    fig.tight_layout()
    save_figure(fig, f"partA_coverage_vs_n_{dgp}")
    plt.close(fig)

print("Saved Part A coverage figures.")

In [ ]:
# Part A plots: reject rate vs n by method for each DGP
method_order = ["iid_normal_analytic", "hac_newey_west", "psr", "dsr"]

for dgp in PARTA_DGPS:
    s_values = list(CONFIG["S_true_list"])
    fig, axes = plt.subplots(1, len(s_values), figsize=(6 * len(s_values), 4), sharey=True)
    if len(s_values) == 1:
        axes = [axes]

    for ax, s_true in zip(axes, s_values):
        sub = results_partA[(results_partA["dgp"] == dgp) & (results_partA["S_true"] == float(s_true))]
        for method in method_order:
            s = sub[sub["method"] == method].sort_values("n")
            y = s["reject_rate_H0_S_le_0"].to_numpy(dtype=float)
            if np.isfinite(y).any():
                ax.plot(s["n"], y, marker="o", label=method)

        ax.set_title(f"{dgp} | S_true={s_true}")
        ax.set_xlabel("n")
        ax.set_ylabel("reject_rate_H0_S_le_0")
        ax.set_xticks(CONFIG["n_list"])
        ax.set_ylim(0.0, 1.0)

    handles, labels = axes[0].get_legend_handles_labels()
    if handles:
        fig.legend(handles, labels, loc="upper center", ncol=min(4, len(labels)))
    fig.suptitle(f"Part A Reject Rate vs n ({dgp})", y=1.05)
    fig.tight_layout()
    save_figure(fig, f"partA_reject_vs_n_{dgp}")
    plt.close(fig)

print("Saved Part A reject-rate figures.")

In [ ]:
# Part B: load data and split into train/holdout
partB_status = {
    "run": False,
    "reason": "",
}

partB_window_records = []
partB_diag_df = pd.DataFrame()
partB_results = pd.DataFrame()
model_fit_summary = {}

PARTB_N = 240


def _load_excess_return_series(path: Path) -> pd.Series:
    if path.suffix.lower() in {".parquet", ".pq"}:
        obj = pd.read_parquet(path)
    else:
        obj = pd.read_csv(path)

    if isinstance(obj, pd.Series):
        s = obj
    elif isinstance(obj, pd.DataFrame):
        num_cols = list(obj.select_dtypes(include=[np.number]).columns)
        if not num_cols:
            raise ValueError("No numeric return column found in input file.")
        preferred = [c for c in num_cols if c.lower() in {"excess_return", "excess", "rx", "ret_excess", "return"}]
        use_col = preferred[0] if preferred else num_cols[0]
        s = obj[use_col]

        # If a date-like column exists, use it as index for friendlier window labels.
        date_candidates = [c for c in obj.columns if "date" in c.lower() or "month" in c.lower()]
        if date_candidates:
            try:
                idx = pd.to_datetime(obj[date_candidates[0]], errors="coerce")
                if idx.notna().sum() >= int(0.7 * len(obj)):
                    s = pd.Series(s.to_numpy(), index=idx)
            except Exception:
                pass
    else:
        raise ValueError("Unsupported data object type.")

    s = pd.to_numeric(s, errors="coerce").dropna()
    return s.astype(float)


data_path = Path(CONFIG["data_path"])
if not data_path.exists():
    partB_status["reason"] = f"Skipped Part B: data_path not found -> {data_path}"
    print(partB_status["reason"])
else:
    try:
        returns = _load_excess_return_series(data_path)
    except Exception as e:
        partB_status["reason"] = f"Skipped Part B: failed to load/parse data ({e})"
        print(partB_status["reason"])
        returns = None

    if returns is not None:
        T = int(len(returns))
        if T < 2 * PARTB_N:
            partB_status["reason"] = f"Skipped Part B: need at least {2 * PARTB_N} observations, got {T}."
            print(partB_status["reason"])
        else:
            holdout_len = 360 if T >= 1200 else max(PARTB_N, int(round(0.30 * T)))
            train_len = T - holdout_len
            if train_len < PARTB_N:
                partB_status["reason"] = (
                    f"Skipped Part B: training length too short ({train_len}) after split; "
                    f"T={T}, holdout_len={holdout_len}."
                )
                print(partB_status["reason"])
            else:
                train = returns.iloc[:train_len].copy()
                holdout = returns.iloc[train_len:].copy()
                n_windows = int(len(holdout) // PARTB_N)
                if n_windows < 1:
                    partB_status["reason"] = (
                        f"Skipped Part B: holdout has {len(holdout)} points, no disjoint window of n={PARTB_N}."
                    )
                    print(partB_status["reason"])
                else:
                    partB_status["run"] = True
                    partB_status["reason"] = "Part B ready."
                    holdout_use = holdout.iloc[: n_windows * PARTB_N]

                    for j in range(n_windows):
                        w = holdout_use.iloc[j * PARTB_N : (j + 1) * PARTB_N]
                        s_obs, _, _ = sharpe_mc.sharpe_ratio(w.to_numpy(dtype=np.float64))

                        start_label = w.index[0] if len(w.index) else j * PARTB_N
                        end_label = w.index[-1] if len(w.index) else (j + 1) * PARTB_N - 1

                        partB_window_records.append(
                            {
                                "window_id": j,
                                "window_start": str(start_label),
                                "window_end": str(end_label),
                                "S_obs": float(s_obs),
                            }
                        )

                    partB_window_obs = pd.DataFrame(partB_window_records)
                    print(
                        f"Part B data loaded: T={T}, train_len={train_len}, holdout_len={len(holdout)}, "
                        f"disjoint_windows={n_windows}, n_window={PARTB_N}."
                    )

In [ ]:
# Part B: fit 3 models, bootstrap Sharpe CDFs, compute PIT and diagnostics, save artifacts
partB_diag_rows = []
partB_window_rows = []

if not partB_status["run"]:
    partB_results = pd.DataFrame(
        columns=[
            "model",
            "window_id",
            "window_start",
            "window_end",
            "S_obs",
            "pit_u",
            "ks_stat",
            "ks_pvalue",
            "pit_score",
            "n_windows",
            "n_boot",
        ]
    )
    model_fit_summary = {"status": "skipped", "reason": partB_status["reason"]}
else:
    train_arr = train.to_numpy(dtype=np.float64)
    train_sig = series_checksum(train_arr)
    Rb = int(CONFIG["R"])

    model_names = ["iid_normal", "iid_t_fixed_nu", "garch11_t_fixed_nu_approx"]
    spawned = np.random.SeedSequence(int(CONFIG["seed"]) + 20260218).spawn(len(model_names))

    for model_idx, model_name in enumerate(model_names):
        model_seed = int(spawned[model_idx].generate_state(1)[0])
        rng = np.random.default_rng(model_seed)

        cache_stem = (
            f"partB_bootstrap_Shat_{model_name}_R{Rb}_n{PARTB_N}_"
            f"seed{model_seed}_nu{_safe_tag(CONFIG['t_df'])}_"
            f"burn{int(CONFIG['burn_B'])}_train{len(train_arr)}_{train_sig}"
        )

        sorted_sr = cache_load_array(cache_stem)
        fit_info = {"model": model_name, "seed": model_seed, "cached": sorted_sr is not None}

        if sorted_sr is None:
            sr_boot = np.empty(Rb, dtype=np.float64)

            if model_name == "iid_normal":
                mu_hat = float(np.mean(train_arr))
                sigma_hat = float(np.std(train_arr, ddof=1))
                fit_info.update({"mu_hat": mu_hat, "sigma_hat": sigma_hat})

                batch = 512
                ptr = 0
                for start in tqdm(range(0, Rb, batch), desc=f"Part B bootstrap {model_name}", leave=False):
                    m = min(batch, Rb - start)
                    draws = mu_hat + sigma_hat * rng.standard_normal(size=(m, PARTB_N))
                    means = draws.mean(axis=1)
                    stds = draws.std(axis=1, ddof=1)
                    sr_boot[ptr : ptr + m] = means / stds
                    ptr += m

            elif model_name == "iid_t_fixed_nu":
                nu = float(CONFIG["t_df"])
                scale = np.sqrt(nu / (nu - 2.0))
                mu_hat = float(np.mean(train_arr))
                sigma_hat = float(np.std(train_arr, ddof=1))
                fit_info.update({"mu_hat": mu_hat, "sigma_hat": sigma_hat, "nu_fixed": nu})

                batch = 512
                ptr = 0
                for start in tqdm(range(0, Rb, batch), desc=f"Part B bootstrap {model_name}", leave=False):
                    m = min(batch, Rb - start)
                    z = rng.standard_t(df=nu, size=(m, PARTB_N)) / scale
                    draws = mu_hat + sigma_hat * z
                    means = draws.mean(axis=1)
                    stds = draws.std(axis=1, ddof=1)
                    sr_boot[ptr : ptr + m] = means / stds
                    ptr += m

            elif model_name == "garch11_t_fixed_nu_approx":
                # Closest project-native approach: fit garch11_t with project fitter (nu estimated),
                # then set nu to CONFIG['t_df'] for simulation if fixed-nu fitting is unavailable.
                model_obj, fit_res, params_hat = sharpe_mc.fit_candidate(train_arr, "garch11_t")
                params_sim = np.asarray(params_hat, dtype=np.float64).copy()
                nu_hat = float(params_sim[-1])
                params_sim[-1] = float(CONFIG["t_df"])
                initial_var = float(max(np.var(train_arr, ddof=1), 1e-8))

                fit_info.update(
                    {
                        "mu_hat": float(params_sim[0]),
                        "omega_hat": float(params_sim[1]),
                        "alpha_hat": float(params_sim[2]),
                        "beta_hat": float(params_sim[3]),
                        "nu_hat_from_fit": nu_hat,
                        "nu_used_in_sim": float(params_sim[-1]),
                        "note": (
                            "Fixed-nu optimization is not directly exposed by the project fitter; "
                            "used closest approach: estimate params with project garch11_t fitter, "
                            "then overwrite nu in simulation."
                        ),
                    }
                )

                # arch simulation inside sharpe_mc uses global np RNG
                set_global_seed(model_seed)
                for i in tqdm(range(Rb), desc=f"Part B bootstrap {model_name}", leave=False):
                    sim = sharpe_mc.simulate_from_fit(
                        model_obj,
                        params_sim,
                        n=PARTB_N,
                        burn=int(CONFIG["burn_B"]),
                        initial_value_vol=initial_var,
                    )
                    s_hat_i, _, _ = sharpe_mc.sharpe_ratio(sim)
                    sr_boot[i] = float(s_hat_i)
            else:
                raise ValueError(model_name)

            sr_boot = sr_boot[np.isfinite(sr_boot)]
            if sr_boot.size == 0:
                raise RuntimeError(f"No finite bootstrap Sharpe draws for model {model_name}.")
            sorted_sr = np.sort(sr_boot)
            cache_save_array(sorted_sr, cache_stem)

        model_fit_summary[model_name] = fit_info

        # PIT for each disjoint holdout window
        u_vals = []
        for _, wrow in partB_window_obs.iterrows():
            s_obs = float(wrow["S_obs"])
            u = float(np.searchsorted(sorted_sr, s_obs, side="right") / sorted_sr.size)
            u_vals.append(u)
            partB_window_rows.append(
                {
                    "model": model_name,
                    "window_id": int(wrow["window_id"]),
                    "window_start": wrow["window_start"],
                    "window_end": wrow["window_end"],
                    "S_obs": s_obs,
                    "pit_u": u,
                }
            )

        u_arr = np.asarray(u_vals, dtype=np.float64)
        if u_arr.size == 0:
            ks_stat, ks_pvalue, pit_score = np.nan, np.nan, np.nan
        else:
            ks = stats.kstest(u_arr, "uniform")
            ks_stat = float(ks.statistic)
            ks_pvalue = float(ks.pvalue)
            pit_score = float(np.mean(np.abs(u_arr - 0.5)))

        partB_diag_rows.append(
            {
                "model": model_name,
                "ks_stat": ks_stat,
                "ks_pvalue": ks_pvalue,
                "pit_score": pit_score,
                "n_windows": int(u_arr.size),
                "n_boot": int(sorted_sr.size),
            }
        )

    partB_window_df = pd.DataFrame(partB_window_rows)
    partB_diag_df = pd.DataFrame(partB_diag_rows)
    partB_results = partB_window_df.merge(partB_diag_df, on="model", how="left")

# Save Part B outputs (including skipped placeholder if skipped)
partB_paths = result_save_df(partB_results, "results_partB_pit")
fit_summary_path = RESULTS_DIR / "model_fit_summary.json"
fit_summary_path.write_text(json.dumps(model_fit_summary, indent=2), encoding="utf-8")
_track_artifact(fit_summary_path)

print("Saved Part B results:")
for p in partB_paths:
    print("-", p)
print("-", fit_summary_path)

In [ ]:
# Part B diagnostics table
if partB_status["run"]:
    print("Part B diagnostics:")
    print(partB_diag_df)
else:
    print(partB_status["reason"])

print("\nPart B window-level head:")
print(partB_results.head())


In [ ]:
# Part B plots: PIT histograms per model
if partB_results.empty:
    print("Skipping PIT histogram plots because Part B results are empty.")
else:
    for model_name, sub in partB_results.groupby("model"):
        u = sub["pit_u"].to_numpy(dtype=np.float64)
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.hist(u, bins=np.linspace(0.0, 1.0, 11), density=True, alpha=0.75, edgecolor="black")
        ax.axhline(1.0, color="black", linestyle="--", linewidth=1.0)
        ax.set_xlim(0.0, 1.0)
        ax.set_xlabel("PIT u")
        ax.set_ylabel("density")
        ax.set_title(f"PIT histogram | {model_name}")
        fig.tight_layout()
        save_figure(fig, f"partB_pit_hist_{model_name}")
        plt.close(fig)

    print("Saved Part B PIT histogram figures.")

In [ ]:
# Part B plots: ECDF(u) vs uniform line per model
if partB_results.empty:
    print("Skipping PIT ECDF plots because Part B results are empty.")
else:
    for model_name, sub in partB_results.groupby("model"):
        u = np.sort(sub["pit_u"].to_numpy(dtype=np.float64))
        m = u.size
        y = np.arange(1, m + 1, dtype=np.float64) / m

        fig, ax = plt.subplots(figsize=(6, 4))
        ax.step(u, y, where="post", label="ECDF(u)")
        ax.plot([0, 1], [0, 1], linestyle="--", color="black", label="Uniform(0,1)")
        ax.set_xlim(0.0, 1.0)
        ax.set_ylim(0.0, 1.0)
        ax.set_xlabel("u")
        ax.set_ylabel("ECDF")
        ax.set_title(f"PIT ECDF vs Uniform | {model_name}")
        ax.legend(loc="lower right")
        fig.tight_layout()
        save_figure(fig, f"partB_pit_ecdf_{model_name}")
        plt.close(fig)

    print("Saved Part B ECDF figures.")

In [ ]:
# Final run summary
print("Part A results (head):")
print(results_partA.head())

print("\nPart B results (head):")
print(partB_results.head())

print("\nSaved artifacts:")
for p in sorted(set(SAVED_ARTIFACTS)):
    print("-", p)
